**20L-0961**

We have a huge data file of FAST students. The file consists of the roll number of
the FAST students, followed by the subject code and the student grade in that subject. <br>
Consider an input file in the following format

In [1]:
%%writefile grades.txt
L22-2100 DB D
K21-1601 SE F
I21-1601 OS F
K21-1702 DS B
L21-1705 OS A
L22-2101 DB D
K21-1601 OS F
L21-1601 SE F
L21-1702 SE B
L21-1705 DB A

Overwriting grades.txt


## Input the data from the text file in an RDD

In [1]:
import findspark
findspark.init()
findspark.find()

'c:\\Users\\ammar\\anaconda3\\Lib\\site-packages\\pyspark'

In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc=SparkContext.getOrCreate()
spark=SparkSession(sc)
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [5]:
# Read data from 'grades.txt'
data = spark.read.text("grades.txt").rdd.map(lambda x: x[0])

# Process data to extract roll numbers, subjects, and grades
rdd = data.map(lambda x: x.split())

rdd.collect()

[['L22-2100', 'DB', 'D'],
 ['K21-1601', 'SE', 'F'],
 ['I21-1601', 'OS', 'F'],
 ['K21-1702', 'DS', 'B'],
 ['L21-1705', 'OS', 'A'],
 ['L22-2101', 'DB', 'D'],
 ['K21-1601', 'OS', 'F'],
 ['L21-1601', 'SE', 'F'],
 ['L21-1702', 'SE', 'B'],
 ['L21-1705', 'DB', 'A']]

i. Select the records of students from the Lahore campus. Display a few records and print the count of the
students from Lahore.

In [6]:
# Filter records for students from Lahore
lahore_students = rdd.filter(lambda x: x[0].startswith('L'))

lahore_students.collect()


[['L22-2100', 'DB', 'D'],
 ['L21-1705', 'OS', 'A'],
 ['L22-2101', 'DB', 'D'],
 ['L21-1601', 'SE', 'F'],
 ['L21-1702', 'SE', 'B'],
 ['L21-1705', 'DB', 'A']]

In [7]:
# Find the total number of students from Lahore
print(lahore_students.count())

6


ii. Filter the records Of the students from the year in the range Of 1995-2018.

In [8]:
# Filter the records Of the students from the year in the range Of 1995-2018.
year_range_students = rdd.filter(lambda x: int(x[0][1:3]) >= 95 and int(x[0][1:3]) <= 18)
year_range_students.collect()

[]

iii. Display the count of students on each Campus.

In [9]:
lhr_campus = rdd.filter(lambda x: 'L' in x[0])
print('Lhr campus count:',lhr_campus.count())
isb_campus = rdd.filter(lambda x: 'I' in x[0])
print('Isb campus count:',isb_campus.count())
khi_campus = rdd.filter(lambda x: 'K' in x[0])
print('Khi campus count:',khi_campus.count())

Lhr campus count: 6
Isb campus count: 1
Khi campus count: 3


iv. Partition the input data on the base of Campus. (override Spark Partitioner).

In [10]:
# work on original data
# map: splitting the data into key-value pairs k=x[0], v=x
# partitionBy: partitioning the data by key, 3 partitions
partitioned_data = data.map(lambda x: (x.split()[0], x)).partitionBy(3)

partitioned_data.collect()

[('K21-1601', 'K21-1601 SE F'),
 ('K21-1702', 'K21-1702 DS B'),
 ('K21-1601', 'K21-1601 OS F'),
 ('L22-2100', 'L22-2100 DB D'),
 ('I21-1601', 'I21-1601 OS F'),
 ('L21-1705', 'L21-1705 OS A'),
 ('L22-2101', 'L22-2101 DB D'),
 ('L21-1601', 'L21-1601 SE F'),
 ('L21-1702', 'L21-1702 SE B'),
 ('L21-1705', 'L21-1705 DB A')]

v. For each course, print the number of failures on each Campus.

In [11]:
# extract grade and course from the data
course_grades = rdd.map(lambda x: (x[2], x[1]))
course_grades.collect()

[('D', 'DB'),
 ('F', 'SE'),
 ('F', 'OS'),
 ('B', 'DS'),
 ('A', 'OS'),
 ('D', 'DB'),
 ('F', 'OS'),
 ('F', 'SE'),
 ('B', 'SE'),
 ('A', 'DB')]

In [12]:
# filter out passing grades
failures = course_grades.filter(lambda x: x[0] not in ['A', 'B', 'C', 'D'])
failures.collect()

[('F', 'SE'), ('F', 'OS'), ('F', 'OS'), ('F', 'SE')]

In [13]:
# Count the number of failures for each course
failures_count_by_course = {}
for grade, course in failures.collect():
    if course in failures_count_by_course:
        failures_count_by_course[course] += 1
    else:
        failures_count_by_course[course] = 1

# Print the results
for course, count in failures_count_by_course.items():
    print(f"Course {course}: {count} failures")

Course SE: 2 failures
Course OS: 2 failures


vi.
Remove all the duplicate rows from input data.

In [14]:
# work on original input data
remove_duplicate = data.distinct()
remove_duplicate.collect()

['L22-2100 DB D',
 'K21-1601 SE F',
 'I21-1601 OS F',
 'K21-1702 DS B',
 'L21-1705 OS A',
 'L22-2101 DB D',
 'K21-1601 OS F',
 'L21-1601 SE F',
 'L21-1702 SE B',
 'L21-1705 DB A']

vii.
Find the minimum and maximum grades in each subject. The ordering of grades is as follows A >B >C >D >F.

In [15]:
# extract grade and course from the data
course_grades = rdd.map(lambda x: (x[2], x[1]))
course_grades.collect()

[('D', 'DB'),
 ('F', 'SE'),
 ('F', 'OS'),
 ('B', 'DS'),
 ('A', 'OS'),
 ('D', 'DB'),
 ('F', 'OS'),
 ('F', 'SE'),
 ('B', 'SE'),
 ('A', 'DB')]

In [16]:
# first element of the tuple is the course, second element is the grade, replicated
min_max_grades = course_grades.map(lambda x: (x[1], (x[0], x[0])))
# reduceByKey: for each key, find the minimum and maximum values, here we compare x and y in whole course_grades rdd
min_max_grades = min_max_grades.reduceByKey(lambda x,y: (min(x[0], y[0]), max(x[1], y[1])))
min_max_grades.collect()

[('DB', ('A', 'D')),
 ('SE', ('B', 'F')),
 ('OS', ('A', 'F')),
 ('DS', ('B', 'B'))]

viii.
We wish to sort the file based on the roll number (hint work with sortByKey ). The two roll-numbers are
compared using the following rule

1. For Campus use lexicographic ordering that iS F < I < K < L < P
2. For year follow the rule of year 16<17 and 99 < 01
3. For the last part of roll-number, follow int ordering.

In [17]:
def custom_sort_key(roll_number):
    # split the roll number into campus, year, and last part of the roll number
    parts = roll_number.split('-')
    campus_order = {'F': 0, 'I': 1, 'K': 2, 'L': 3, 'P': 4}
    year_order = {'16': 0, '17': 1, '99': 2, '01': 3}
    return (
        campus_order.get(parts[0][0], 99),  # Campus
        year_order.get(parts[0][1:], 99),  # Year
        int(parts[1])  # Last part of the roll number
    )
sorted_gpa_data = rdd.sortByKey(keyfunc=custom_sort_key)

# Show the sorted data
sorted_gpa_data.collect()

[['I21-1601', 'OS', 'F'],
 ['K21-1601', 'SE', 'F'],
 ['K21-1601', 'OS', 'F'],
 ['K21-1702', 'DS', 'B'],
 ['L21-1601', 'SE', 'F'],
 ['L21-1702', 'SE', 'B'],
 ['L21-1705', 'OS', 'A'],
 ['L21-1705', 'DB', 'A'],
 ['L22-2100', 'DB', 'D'],
 ['L22-2101', 'DB', 'D']]

ix.For each student, compute the GPA. Assume only five grades (Grade A GPA=4, Grade B GPA=3, Grade C GPA
2, Grade D GPA 1, and Grade F GPA=0)

In [18]:
# Define a function to map grades to GPA
def grade_to_gpa(grade):
    grade_mapping = {'A': 4, 'B': 3, 'C': 2, 'D': 1, 'F': 0}
    return grade_mapping.get(grade)

# Custom map with first 2 parts as is and grade mapped to GPA
gpa_data = rdd.map(lambda x: (x[0], x[1], grade_to_gpa(x[2])))

# Display the calculated GPA
gpa_data.collect()

[('L22-2100', 'DB', 1),
 ('K21-1601', 'SE', 0),
 ('I21-1601', 'OS', 0),
 ('K21-1702', 'DS', 3),
 ('L21-1705', 'OS', 4),
 ('L22-2101', 'DB', 1),
 ('K21-1601', 'OS', 0),
 ('L21-1601', 'SE', 0),
 ('L21-1702', 'SE', 3),
 ('L21-1705', 'DB', 4)]

x.
Convert grades to GPA as mentioned in part viii and find the average GPA of each Subject

In [19]:
gpa_data = rdd.map(lambda x: (x[0], x[1], grade_to_gpa(x[2])))

# Group by Subject and calculate the average GPA
average_gpa_data = gpa_data.groupBy(lambda x: x[1]).mapValues(lambda x: sum(val[2] for val in x) / len(x))
average_gpa_data.collect()

[('DB', 2.0), ('SE', 1.0), ('OS', 1.3333333333333333), ('DS', 3.0)]